**Import Packages**

In [2]:
import numpy as np

In [3]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [4]:
# LangGraph 架構
from langgraph.graph import StateGraph, END
from typing import TypedDict

**Define Knowledge Base**

In [10]:
# 目前知識庫
# docs_text = """
# 火影代數	姓名	師傅	徒弟
# 初代	千手柱間	無明確記載	猿飛日斬、水戶門炎、轉寢小春
# 二代	千手扉間	千手柱間（兄長）	猿飛日斬、志村團藏、宇智波鏡等
# 三代	猿飛日斬	千手柱間、千手扉間	自來也、大蛇丸、千手綱手（傳說三忍）
# 四代	波風湊	自來也	旗木卡卡西、宇智波帶土、野原琳
# 五代	千手綱手	猿飛日斬	春野櫻、志乃等（主要為春野櫻）
# 六代	旗木卡卡西	波風湊	漩渦鳴人、宇智波佐助、春野櫻（第七班）
# 七代	漩渦鳴人	自來也、旗木卡卡西	木葉丸等（主要為木葉丸）
# """

In [13]:
docs_text = '''
    "初代火影是千手柱間，沒有明確的師傅，他的徒弟是猿飛日斬、水戶門炎與轉寢小春。",
    "二代火影是千手扉間，是千手柱間的弟弟，他的徒弟有猿飛日斬、志村團藏與宇智波鏡。",
    "三代火影是猿飛日斬，師承千手柱間與千手扉間，徒弟是自來也、大蛇丸與千手綱手（傳說三忍）。",
    "四代火影是波風湊，師傅是自來也，他的徒弟有旗木卡卡西、宇智波帶土與野原琳。",
    "五代火影是千手綱手，師傅是猿飛日斬，她的徒弟主要是春野櫻，還有志乃。",
    "六代火影是旗木卡卡西，師傅是波風湊，他的徒弟是漩渦鳴人、宇智波佐助與春野櫻（第七班）。",
    "七代火影是漩渦鳴人，他的師傅是自來也與旗木卡卡西，他的徒弟是木葉丸。"
'''

**Load Models**

In [6]:
embed_model = SentenceTransformer('shibing624/text2vec-base-chinese')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [21]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen1.5-0.5B-Chat", trust_remote_code=True)
chat_model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen1.5-0.5B-Chat", trust_remote_code=True)

In [17]:
doc_embeddings = embed_model.encode(docs_text)

**Excute Chatbot**

In [50]:
# ✅ 狀態定義
class AppState(TypedDict):
    user_query: str
    rag_trigger: bool
    docs: str
    answer: str
    mode: str  # 新增模式字段

# ✅ 節點1：Embedding + 相似度分流
def embed_node(state: AppState) -> AppState:
    query = state["user_query"]

    query_vec = np.atleast_2d(embed_model.encode(query))  # Query encoding
    doc_embeddings_2d = np.atleast_2d(np.asarray(doc_embeddings))  # Doc embeddings

    similarities = cosine_similarity(query_vec, doc_embeddings_2d)[0]

    # 設定相似度閾值為n若任何文檔的相似度大於閾值，啟動 RAG 模式
    rag_trigger = any(similarity > 0.6 for similarity in similarities)

    # 返回相似度大於n的文檔
    matched = "\n".join([docs_text[i] for i, similarity in enumerate(similarities) if similarity > 0.6])

    return {**state, "rag_trigger": rag_trigger, "docs": matched, "mode": "knowledge_base" if rag_trigger else "chat"}

# ✅ 節點2：決策 router（根據查詢決定是否使用知識庫檢索或普通聊天）
def decision_node(state: AppState) -> AppState:
    # 根據 rag_trigger 決定下一步處理節點
    if state["rag_trigger"]:
        return {**state, "mode": "knowledge_base"}
    else:
        return {**state, "mode": "chat"}

# ✅ 節點3：RAG 回答模式
def rag_node(state: AppState) -> AppState:
    query = state["user_query"]
    matched_knowledge = state["docs"]
    print("🔍 匹配知識：\n", matched_knowledge)

    # 🧠 改良後的提示語
    rag_prompt = f"""你是一位火影忍者的知識專家。以下是來自知識庫的資料，內容包含火影角色的師徒關係。
請你**只根據資料回答問題**，**不要補充問題中未問及的其他資訊**。禁止編造或猜測。
請明確斷誰是火影，以及該火影的師傅是誰，以及該夥影的徒弟有哪些人

如果找不到答案，請回答「我不知道」。

【資料】
{matched_knowledge}

【問題】
{query}

請簡潔回答，**只針對問題內容做出回應，不多說。**"""

    messages = [
        {"role": "system", "content": "你是一位火影知識助手，僅能根據提供的資料回答問題，禁止捏造。"},
        {"role": "user", "content": rag_prompt}
    ]

    prompt_text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    model_inputs = tokenizer([prompt_text], return_tensors="pt")

    with torch.no_grad():
        generated_ids = chat_model.generate(**model_inputs, max_new_tokens=256)

    output = tokenizer.batch_decode(generated_ids[:, model_inputs.input_ids.shape[1]:], skip_special_tokens=True)[0]

    return {**state, "answer": output.strip(), "mode": "knowledge_base", "next_node": "END"}


# ✅ 節點4：一般聊天模式
def chat_answer_node(state: AppState) -> AppState:
    query = state["user_query"]

    messages = [
        {"role": "system", "content": "你是一位親切的助理"},
        {"role": "user", "content": query}
    ]
    prompt_text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer([prompt_text], return_tensors="pt")

    with torch.no_grad():
        generated_ids = chat_model.generate(**inputs, max_new_tokens=256)
    output = tokenizer.batch_decode(generated_ids[:, inputs.input_ids.shape[1]:], skip_special_tokens=True)[0]
    return {**state, "answer": output, "mode": "chat", "next_node": "END"}

# ✅ 節點5：路由節點（根據 decision_node 的結果決定調用哪個節點）
def route_node(state: AppState) -> AppState:
    if state["mode"] == "knowledge_base":
        return rag_node(state)  # 如果是知識庫檢索模式
    else:
        return chat_answer_node(state)  # 普通聊天模式

# ✅ 建立 LangGraph
workflow = StateGraph(AppState)
workflow.add_node("embed", embed_node)
workflow.add_node("decision", decision_node)
workflow.add_node("route", route_node)  # 新增路由節點
workflow.add_node("__rag__", rag_node)
workflow.add_node("__chat__", chat_answer_node)
workflow.set_entry_point("embed")
workflow.add_edge("embed", "decision")
workflow.add_edge("decision", "route")  # 由 decision 進行路由
workflow.add_edge("route", END)  # 路由結束
app = workflow.compile()

# ✅ 互動測試迴圈
print("🔁 啟動互動模式，輸入 '結束' 結束對話")
while True:
    query = input("🧑 您：")
    if query.strip() in ["exit", "結束", "quit"]:
        print("👋 再見！")
        break
    result = app.invoke({"user_query": query})
    print(f"🤖 機器人（模式: {result['mode']}）：{result['answer']}")

🔁 啟動互動模式，輸入 '結束' 結束對話
🧑 您：第四代火影是誰
🔍 匹配知識：
 四代火影是波風湊，師傅是自來也，他的徒弟有旗木卡卡西、宇智波帶土與野原琳。
🤖 機器人（模式: knowledge_base）：我知道，第四代火影是波風湊。
🧑 您：第四代火影的徒弟有哪些人
🔍 匹配知識：
 初代火影是千手柱間，沒有明確的師傅，他的徒弟是猿飛日斬、水戶門炎與轉寢小春。
四代火影是波風湊，師傅是自來也，他的徒弟有旗木卡卡西、宇智波帶土與野原琳。
五代火影是千手綱手，師傅是猿飛日斬，她的徒弟主要是春野櫻，還有志乃。
六代火影是旗木卡卡西，師傅是波風湊，他的徒弟是漩渦鳴人、宇智波佐助與春野櫻（第七班）。
七代火影是漩渦鳴人，他的師傅是自來也與旗木卡卡西，他的徒弟是木葉丸。
🤖 機器人（模式: knowledge_base）：旗木卡卡西、宇智波帶土與野原琳為第四代火影的徒弟。
🧑 您：第七代火影是誰
🔍 匹配知識：
 六代火影是旗木卡卡西，師傅是波風湊，他的徒弟是漩渦鳴人、宇智波佐助與春野櫻（第七班）。
七代火影是漩渦鳴人，他的師傅是自來也與旗木卡卡西，他的徒弟是木葉丸。
🤖 機器人（模式: knowledge_base）：漩渦鳴人
🧑 您：你好，你是什麼模型
🤖 機器人（模式: chat）：我是一個聊天機台，我叫通義千問。我可以回答問題、提供信息、聊天等各種任務，請隨時问我任何問題！
🧑 您：結束
👋 再見！
